## Back Propagation 구현

In [29]:
import time
import numpy as np

### 1. define function

In [35]:
epsilon = 0.0001

def _t(x):
    return np.transpose(x)

def _m(A,B):
    return np.matmul(A,B)

def sigmoid(x):
    return 1/(1+np.exp(-x))

def mse(h,y):
    return 1/(2*np.mean(np.square(h-y)))

### 2. Neuron

In [36]:
class Neuron:
    def __init__(self,W,b,a):
        # Model Parameter
        self.W = W
        self.b = b
        self.a = a
        
        # Gradients
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)
    
    def __call__(self, x):
        return self.a(_m(_t(self.W),x) + self.b)

### 3. DNN ( Deep Neural Network )

In [45]:
class DNN:
    def __init__(self,hidden_depth, num_neuron, num_input, num_output, activation=sigmoid):
        def init_var(i,o):
            return np.random.normal(0.0, 0.01, (i,o)), np.zeros((o,)) # initialize W & b
        
        self.sequence = list()
        
        # 1st Hidden Layer
        W,b = init_var(num_input,num_neuron)
        self.sequence.append(Neuron(W,b,activation))
        
        # 2nd~last Hidden Layer
        for _ in range(hidden_depth - 1):
            W,b = init_var(num_neuron,num_neuron)
            self.sequence.append(Neuron(W,b,activation))
        
        # Output Layer
        W,b = init_var(num_neuron,num_output)
        self.sequence.append(Neuron(W,b,activation))
        
    def __call__(self,x):
        for layer in self.sequence:
            x = layer(x)
        return x
    
    def calc_grad(self,x,y,loss_func):
        def get_new_sequence(layer_index, new_neuron):
            new_sequence = list()
            for i,layer in enumerate(self.sequence):
                if i == layer_index :
                    new_sequence.append(new_neuron) # change
                else :
                    new_sequence.append(layer) # not change
            return new_sequence
        
        def eval_sequence(x,sequence):
            for layer in sequence:
                x = layer(x)
            return x
        
        loss = loss_func(self(x), y)
        
        for layer_id, layer in enumerate(self.sequence):
            for w_i, w in enumerate(layer.W): # iterate W (row)
                for w_j, ww in enumerate(w): # iterate W (col)
                    W = np.copy(layer.W)
                    W[w_i][w_j] = ww + epsilon
                    
                    new_neuron = Neuron(W,layer.b,layer.a)
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    h = eval_sequence(x,new_seq)
                    
                    num_grad = (loss_func(h,y) - loss) / epsilon # f(x+eps)-f(x) / eps
                    layer.dW[w_i][w_j] = num_grad
                
                for b_i, bb in enumerate(layer.b):
                    b = np.copy(layer.b)
                    b[b_i] = bb + epsilon
                    
                    new_neuron = Neuron(layer.W, b,layer.a)
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    h = eval_sequence(x,new_seq)
                    
                    num_grad = (loss_func(h,y) - loss) / epsilon # f(x+eps)-f(x) / eps
                    layer.db[b_i] = num_grad
                    
            return loss                
                    

### 4. Gradient Descent

In [46]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = network.calc_grad(x,y,loss_obj)
    for layer in network.sequence:
        layer.W += -alpha*layer.dW
        layer.b += -alpha*layer.db
    return loss

In [47]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0,1.0, (2,))

dnn = DNN(hidden_depth=5, num_neuron=32, num_input=10, num_output=2, activation=sigmoid)

t = time.time()
for epoch in range(100):
    loss = gradient_descent(dnn,x,y,mse,0.01)
    print('Epoch {} : Test Loss {}'.format(epoch, loss))

print('{} seconds elapsed.'.format(time.time()-t))

Epoch 0 : Test Loss 2.2755415869729543
Epoch 1 : Test Loss 2.2755415869729543
Epoch 2 : Test Loss 2.2755415869729543
Epoch 3 : Test Loss 2.2755415869729543
Epoch 4 : Test Loss 2.2755415869729543
Epoch 5 : Test Loss 2.2755415869729543
Epoch 6 : Test Loss 2.2755415869729543
Epoch 7 : Test Loss 2.2755415869729543
Epoch 8 : Test Loss 2.2755415869729543
Epoch 9 : Test Loss 2.2755415869729543
Epoch 10 : Test Loss 2.2755415869729543
Epoch 11 : Test Loss 2.2755415869729543
Epoch 12 : Test Loss 2.2755415869729543
Epoch 13 : Test Loss 2.2755415869729543
Epoch 14 : Test Loss 2.2755415869729543
Epoch 15 : Test Loss 2.2755415869729543
Epoch 16 : Test Loss 2.2755415869729543
Epoch 17 : Test Loss 2.2755415869729543
Epoch 18 : Test Loss 2.2755415869729543
Epoch 19 : Test Loss 2.2755415869729543
Epoch 20 : Test Loss 2.2755415869729543
Epoch 21 : Test Loss 2.2755415869729543
Epoch 22 : Test Loss 2.2755415869729543
Epoch 23 : Test Loss 2.2755415869729543
Epoch 24 : Test Loss 2.2755415869729543
Epoch 25 :

### TOO LONG TIME... Back Propagation을 쓸 경우?